In [3]:
# делаем import необходимых библиотек
import os
import sys

import mlflow

/home/mle-user/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/mle-user/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [4]:
# !{sys.executable} -m pip install psycopg
# !{sys.executable} -m pip install 'psycopg[binary]'


In [5]:
import psycopg
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

### Запуск с локальной записью артефактов и метрик

In [1]:


# устанавливаем локальное хранилище для наших экспериментов
# хранилище должно быть такое же, как и при запуске сервиса
mlflow.set_tracking_uri('file:./mlflow_experiments_store')

# получаем id эксеримента, который создаётся по умолчанию
# эксперимент по умолчанию называется Default
experiment_id = mlflow.get_experiment_by_name("Default").experiment_id

# залогируем тестовую метрику и артефакт
with mlflow.start_run(run_name='Default', experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_metric("test_metric", 0)
    mlflow.log_artifact("test_artifact.txt", "test_artifact")

print(f"Run id запуска: {run_id}") 

/home/mle-user/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/mle-user/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


Run id запуска: a874309545674637a6f48c6858ff8a7c


In [2]:
!ls mlflow_experiments_store/0/{run_id}/metrics

test_metric


In [3]:
!cat mlflow_experiments_store/0/{run_id}/artifacts/test_artifact/test_artifact.txt

test_artifact

### Запуск с БД sqlite

In [4]:
mlflow.set_tracking_uri('http://0.0.0.0:5000')

# получаем id эксперимента, который создаётся по умолчанию
# эксперимент по умолчанию называется Default
experiment_id = mlflow.get_experiment_by_name("Default").experiment_id

with mlflow.start_run(run_name="Default", experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_metric("test_metric_sqlite", 0)
    mlflow.log_artifact("test_artifact.txt", "test_artifact_sqlite")

# проверим, что наши данные сохранились в локальной папке, а также создалась база данных SQLite
assert os.path.exists("mlflow_experiments_store_sqlite")
assert os.path.exists("mydb.sqlite") 

### Запуск с удаленным хранилищем артефактов

In [3]:
# определяем основные credentials, которые нужны для подключения к MLflow
# важно, что credentials мы передаём для себя как пользователей Tracking Service
# у вас должен быть доступ к бакету, в который вы будете складывать артефакты
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

YOUR_NAME = "alexdem" # введите своё имя для создания уникального эксперимента
assert YOUR_NAME, "введите своё имя в переменной YOUR_NAME для создания уникального эксперимента"

# название тестового эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = f"test_connection_experiment_{YOUR_NAME}"
RUN_NAME = "test_connection_run"

# тестовые данные
METRIC_NAME = "test_metric"
METRIC_VALUE = 0

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# создаём тестовый эксперимент и записываем в него тестовую информацию
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_metric(METRIC_NAME, METRIC_VALUE) 

In [4]:
##### 4. Проверяем себя, что в MLflow:
# - создался `experiment` с нашим именем
# - внутри эксперимента появился запуск `run`
# - внутри `run` записалась наша тестовая `metric`
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
run = mlflow.get_run(run_id)

assert "active" == experiment.lifecycle_stage
assert mlflow.get_run(run_id)
assert METRIC_VALUE == run.data.metrics[METRIC_NAME] 

### Выгрузка данных

In [6]:
import psycopg2 as psycopg
import pandas as pd

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"), 
    "port": "6432",
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определяем название таблицы, в которой хранятся наши данные
TABLE_NAME = "users_churn"


# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций с базой данных
# причём закрыто оно будет даже в случае ошибки при работе с базой данных
# это нужно, чтобы не допустить так называемую "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаём объект курсора для выполнения запросов к базе данных 
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
				
				# извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

				# получаем список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаём объект DataFrame из полученных данных и имён столбцов 
# это позволяет удобно работать с данными в Python с использованием библиотеки Pandas
df = pd.DataFrame(data, columns=columns)

print(f"Размер нашей таблицы: {df.shape[0]} строк; {df.shape[1]} столбцов")

Размер нашей таблицы: 7043 строк; 22 столбцов


In [7]:
# определяем основные credentials, которые нужны для подключения к MLflow
# важно, что credentials мы передаём для себя как пользователей Tracking Service
# у вас должен быть доступ к бакету, в который вы будете складывать артефакты
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

In [8]:
# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [6]:
EXPERIMENT_NAME = "churn_task_alexdem"
RUN_NAME = "data_check"

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment_id is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment_id.experiment_id

In [8]:
experiment_id

'2'

In [11]:
stats = {"mae": 2, "r2": 0.82}
artifact_path = "dataframe"

In [12]:
with open("columns.txt", "w") as f:
    f.write(",".join(df.columns))

    

In [13]:
df.to_csv("users_churn.csv", index=False)

In [7]:

    

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id
    
    # логируем метрики эксперимента
    # предполагается, что переменная stats содержит словарь с метриками,
    # где ключи — это названия метрик, а значения — числовые значения метрик
    mlflow.log_metrics(stats)
    
    # логируем файлы как артефакты эксперимента — 'columns.txt' и 'users_churn.csv'
    mlflow.log_artifact("columns.txt", artifact_path)
    mlflow.log_artifact("users_churn.csv", artifact_path)


experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id)


# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert run.info.status == "FINISHED"

# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов


NameError: name 'RUN_NAME' is not defined

### Логирование метрик

In [9]:
from sklearn.metrics import mean_absolute_error, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, confusion_matrix, precision_score, recall_score, f1_score, log_loss

from catboost import CatBoostClassifier



In [10]:
df.dtypes

id                            int64
customer_id                  object
begin_date           datetime64[ns]
end_date             datetime64[ns]
type                         object
paperless_billing            object
payment_method               object
monthly_charges             float64
total_charges               float64
internet_service             object
online_security              object
online_backup                object
device_protection            object
tech_support                 object
streaming_tv                 object
gender                       object
streaming_movies             object
senior_citizen                int64
partner                      object
dependents                   object
multiple_lines               object
target                        int64
dtype: object

In [25]:
df['target'].value_counts()

target
0    5174
1    1869
Name: count, dtype: int64

In [26]:
df['is_active'] = df['end_date'] == 0


In [37]:
cat_features = df.drop(columns=['end_date']).dtypes[df.dtypes == 'object'].index.tolist()
drop_features = df.dtypes[df.dtypes == 'datetime64[ns]'].index.tolist() + ['end_date']
target = ['target']

In [38]:
df['total_charges'] = df['total_charges'].fillna(0)
df.fillna(0, inplace=True)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=target + drop_features), df[target], test_size=0.2, random_state=42)

model = CatBoostClassifier(iterations=300, cat_features=cat_features, verbose=False)
model.fit(X_train, y_train)
proba = model.predict_proba(X_test)[:, 1]
prediction = model.predict(X_test)


In [15]:
confusion_matrix(y_test, prediction)

array([[920, 108],
       [160, 221]])

In [16]:
_, err1, _, err2 = confusion_matrix(y_test, prediction).ravel()

In [17]:
auc = roc_auc_score(y_test, proba)

In [18]:
confusion_matrix(y_test, prediction, normalize='all').ravel()

array([0.65294535, 0.07665011, 0.11355571, 0.15684883])

In [49]:
confusion_matrix(y_test, prediction).reshape(-1)

array([920, 108, 160, 221])

In [35]:
X_train['multiple_lines'].value_counts()

multiple_lines
No     2679
Yes    2399
Name: count, dtype: int64

In [17]:
logloss = log_loss(y_test, proba)

In [14]:
for col in cat_features:
    X_test[col] = X_test[col].astype('str')

In [15]:
X_test.dtypes

id                     int64
customer_id           object
type                  object
paperless_billing     object
payment_method        object
monthly_charges      float64
total_charges        float64
internet_service      object
online_security       object
online_backup         object
device_protection     object
tech_support          object
streaming_tv          object
gender                object
streaming_movies      object
senior_citizen         int64
partner               object
dependents            object
multiple_lines        object
dtype: object

In [14]:
import os

import mlflow


EXPERIMENT_NAME = "churn_task_alexdem"
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_alexdem"


os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000


pip_requirements = "./requirements.txt"
signature = mlflow.models.infer_signature(X_test.values, prediction)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    model_info = mlflow.catboost.log_model( 
			cb_model=model,
            artifact_path="models",
            registered_model_name=REGISTRY_MODEL_NAME,
            pip_requirements=pip_requirements,
            signature=signature,
            input_example=input_example,
            metadata=metadata,
            await_registration_for=60
		)

Registered model 'churn_model_alexdem' already exists. Creating a new version of this model...
2025/05/30 19:44:01 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: churn_model_alexdem, version 2
Created version '2' of model 'churn_model_alexdem'.


In [22]:
loaded_model = mlflow.catboost.load_model(model_uri=model_info.model_uri)
model_predictions = loaded_model.predict(X_test)

### Логирование другой версии модели

In [40]:
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [41]:
auc = roc_auc_score(y_test, proba)
logloss = log_loss(y_test, proba)
_, err1, _, err2 = confusion_matrix(y_test, prediction).ravel()

In [44]:
EXPERIMENT_NAME = "churn_task_alexdem"
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_alexdem"

pip_requirements= "./requirements.txt"
signature = mlflow.models.infer_signature(X_test.values, prediction)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    model_info = mlflow.catboost.log_model( 
			cb_model=model,
            artifact_path="models",
            registered_model_name=REGISTRY_MODEL_NAME,
            pip_requirements=pip_requirements,
            signature=signature,
            input_example=input_example,
            metadata=metadata,
            await_registration_for=60
		)
    mlflow.log_metrics({'auc': auc, 'logloss': logloss, 'err1': err1, 'err2': err2})

/home/mle-user/.local/lib/python3.10/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/mle-user/.local/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'churn_model_alexdem' already exists. Creating a new version of this model...
2025/06/01 15:48:00 INFO mlflow.tracking._mode

In [45]:
client = mlflow.MlflowClient()


models = client.search_model_versions(
    filter_string=f"name = '{REGISTRY_MODEL_NAME}'"
)
print(f"Model info:\n {models}")

model_name_1 = models[-1].name
model_version_1 = models[-1].version
model_stage_1 = models[-1].current_stage

model_name_2 = models[-2].name
model_version_2 = models[-2].version
model_stage_2 = models[-2].current_stage

Model info:
 [<ModelVersion: aliases=[], creation_timestamp=1748792880355, current_stage='None', description='', last_updated_timestamp=1748792880355, name='churn_model_alexdem', run_id='06b7771931c14c828a881f9591c00430', run_link='', source='s3://s3-student-mle-20250329-2e5319257f/2/06b7771931c14c828a881f9591c00430/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='4'>, <ModelVersion: aliases=[], creation_timestamp=1748792731629, current_stage='None', description='', last_updated_timestamp=1748792731629, name='churn_model_alexdem', run_id='d0a545e1cf7d4e6ab55a3cc3af33eea5', run_link='', source='s3://s3-student-mle-20250329-2e5319257f/2/d0a545e1cf7d4e6ab55a3cc3af33eea5/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='3'>, <ModelVersion: aliases=[], creation_timestamp=1748634241316, current_stage='Production', description='', last_updated_timestamp=1748634558267, name='churn_model_alexdem', run_id='30f13eac4e054f0ba05df

In [46]:

client.transition_model_version_stage(
    name=REGISTRY_MODEL_NAME,
    version=4,
    stage="Staging"
)
client.transition_model_version_stage(
    name=REGISTRY_MODEL_NAME,
    version=3,
    stage="Production"
)

    

<ModelVersion: aliases=[], creation_timestamp=1748792731629, current_stage='Production', description='', last_updated_timestamp=1748793204118, name='churn_model_alexdem', run_id='d0a545e1cf7d4e6ab55a3cc3af33eea5', run_link='', source='s3://s3-student-mle-20250329-2e5319257f/2/d0a545e1cf7d4e6ab55a3cc3af33eea5/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='3'>

In [52]:
client.rename_registered_model(
    name=REGISTRY_MODEL_NAME,
    new_name=REGISTRY_MODEL_NAME+"_b2c"
)

